In [ ]:
import pandas as pd
import numpy as np
import datetime
dest = '/content/'
#-------------------------------------------------------------#
#- Step 1 - create list of us Chicago Weather stations codes -#
#-------------------------------------------------------------#
Chicago_Stations = pd.read_csv(dest + 'Chicago_Wards_Stations.csv', header=0 ,usecols=[0], names=['StationCode'])['StationCode'].tolist()
print(Chicago_Stations[:5])
print(len(Chicago_Stations))

In [ ]:
#-----------------------------------------------------------------------#
#- Step 2 - extract from the raw Climate file only Chicago information -#
#-----------------------------------------------------------------------#
VarDict = {'StationCode': str, 'Date':int, 'Flag': str, 'Observation': int}

# - read climate data -#
chunk = pd.read_csv(dest + '2021.csv', header=None, chunksize=300000,
                    usecols=[0,1,2,3], names=['StationCode', 'Date', 'Flag', 'Observation'], dtype=VarDict)
ClimDat = pd.concat(df[(df.StationCode.isin(Chicago_Stations))] for df in chunk)
ClimDat.to_csv(dest + 'Chicago_Climate_2021.csv',index=False)

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------#
#- Step 3 - Transpose such that: for each 'date' will have variables with the mean of the indicators from all stations -#
#-----------------------------------------------------------------------------------------------------------------------#
Chicago_Climate_2021_Tran = (ClimDat.melt(['Date', 'Flag', 'Observation'])
        .pivot_table(index='Date',columns='Flag',values='Observation',aggfunc='mean')
        .reset_index()
        .rename_axis(None, axis=1))


In [ ]:
#------------------------------#
#- Step 4 - add more variables -#
#------------------------------#
#- add day of week and month -#
Chicago_Climate_2021_Tran['Date'] = pd.to_datetime(Chicago_Climate_2021_Tran['Date'])
Chicago_Climate_2021_Tran['DayOfWeek'] = Chicago_Climate_2021_Tran['Date'].dt.dayofweek
Chicago_Climate_2021_Tran['Month'] = pd.DatetimeIndex(Chicago_Climate_2021_Tran['Date']).month
#- add federal holidays -#
Holidays_DF = pd.read_csv(dest + 'Holidays.csv',usecols=[0,1])
Holidays_DF['Date'] = pd.to_datetime(Holidays_DF['Date'])
Chicago_Climate_2021_Tran = Chicago_Climate_2021_Tran.merge(Holidays_DF, how='left', on='Date')

In [ ]:
#-----------------------------#
#- Step 5 - data exploration -#
#-----------------------------#
Corr_Mat = Chicago_Climate_2021_Tran.corr()
Corr_Mat.to_csv(dest + 'Chicago_Climate_Corr_Mat.csv',index=False)
#- group 1 plots -#
fig, axs = plt.subplots(3, 2)
axs[0, 0].plot(x, Chicago_Climate_2021_Tran['PRCP'])
axs[0, 0].set_title("Precipitation")
axs[1, 0].plot(x, Chicago_Climate_2021_Tran['SNOW'])
axs[1, 0].set_title("Snowfall")
axs[1, 0].sharex(axs[0, 0])
axs[0, 1].plot(x, Chicago_Climate_2021_Tran['TMAX'])
axs[0, 1].set_title("Maximum temperature")
axs[1, 1].plot(x, Chicago_Climate_2021_Tran['TMIN'])
axs[1, 1].set_title("Minimum temperature")
axs[2, 0].plot(x, Chicago_Climate_2021_Tran['WSF2'])
axs[2, 0].set_title("Fastest 2-minute wind speed")
axs[2, 1].plot(x, Chicago_Climate_2021_Tran['WDF2'])
axs[2, 1].set_title("Direction of fastest 2-minute wind")
fig.tight_layout()

In [ ]:
#-------------------------------------------------------#
#- Step 6 - drop irrelevant and highly correlated vars -#
#-------------------------------------------------------#
DropList = ['AWND','WSF5','WDF5','WESD','WESF','MDPR','DAPR','TOBS','TAVG']
Chicago_Climate_2021_Tran = Chicago_Climate_2021_Tran.drop(columns=DropList)

In [ ]:
#--------------------------------------------#
#- Step 7 - Combine with Chicago crime data -#
#--------------------------------------------#
ChiCrime = pd.read_csv(dest 'ChicagoCrime.csv')
ChiCrime = ChiCrime.dropna()
print(ChiCrime.columns)
ChiCrime['date'] = pd.to_datetime(ChiCrime['date'])
ChiCrime['Date'] = ChiCrime['date'].dt.strftime('%Y-%m-%d')
ChiCrime['Hour'] = ChiCrime.date.map(lambda x: x.hour)
#- drop irrelevant -#
ChiCrime = ChiCrime.drop(columns=['case_number','date','block','iucr','description','arrest', 'domestic',
'district', 'community_area', 'fbi_code', 'x_coordinate','y_coordinate', 'year', 'updated_on', 'latitude', 'longitude','location'])
#- merge the distance to nearest police station -#
Dist_Crime_To_Station = pd.read_csv(dest + 'Dist_Crime_To_Station.csv',usecols=['unique_key','HubDist'])
ChiCrime = ChiCrime.merge(Dist_Crime_To_Station, how='left', on='unique_key')
ChiCrime = ChiCrime.drop(columns=['unique_key'])
#- merge the climate data -#
ChiCrime = ChiCrime.merge(Chicago_Climate_2021_Tran, how='left', on='Date')
ChiCrime = ChiCrime.fillna(0)
ChiCrime.to_csv(dest + 'Chicago_Crime_Final_Data.csv',index=False)